<a href="https://colab.research.google.com/github/Sanjeevkumar2004/AIML-Project/blob/main/Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!git clone https://github.com/narumiruna/efficientnet-pytorch.git


Cloning into 'efficientnet-pytorch'...
remote: Enumerating objects: 876, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 876 (delta 168), reused 208 (delta 121), pack-reused 572 (from 1)
Receiving objects: 100% (876/876), 293.73 KiB | 1.26 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [ ]:
%cd efficientnet-pytorch

/content/efficientnet-pytorch


In [ ]:
!pip install torch torchvision


In [ ]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"venkidu","key":"b161560ae40037cb78d1de2a0f78561a"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d alvarogarciav/dataset-classifier-cat-dog-snake
!unzip dataset-classifier-cat-dog-snake.zip

Dataset URL: https://www.kaggle.com/datasets/alvarogarciav/dataset-classifier-cat-dog-snake
License(s): MIT
 81% 31.0M/38.4M [00:00<00:00, 93.1MB/s]
100% 38.4M/38.4M [00:00<00:00, 86.0MB/s]
Archive:  dataset-classifier-cat-dog-snake.zip
  inflating: dataset/cats/0_0001.jpg  
  inflating: dataset/cats/0_0002.jpg  
  inflating: dataset/cats/0_0003.jpg  
  inflating: dataset/cats/0_0004.jpg  
  inflating: dataset/cats/0_0005.jpg  
  inflating: dataset/cats/0_0006.jpg  
  inflating: dataset/cats/0_0007.jpg  
  inflating: dataset/cats/0_0008.jpg  
  inflating: dataset/cats/0_0009.jpg  
  inflating: dataset/cats/0_0010.jpg  
  inflating: dataset/cats/0_0011.jpg  
  inflating: dataset/cats/0_0012.jpg  
  inflating: dataset/cats/0_0013.jpg  
  inflating: dataset/cats/0_0014.jpg  
  inflating: dataset/cats/0_0015.jpg  
  inflating: dataset/cats/0_0016.jpg  
  inflating: dataset/cats/0_0017.jpg  
  inflating: dataset/cats/0_0018.jpg  
  inflating: dataset/cats/0_0019.jpg  
  inflating: dataset/c

In [ ]:
import os
import zipfile
import requests

dataset_url = 'https://www.kaggle.com/datasets/alvarogarciav/dataset-classifier-cat-dog-snake'
dataset_path = 'dataset-classifier-cat-dog-snake.zip'

if not os.path.exists(dataset_path):
    response = requests.get(dataset_url)
    with open(dataset_path, 'wb') as file:
        file.write(response.content)

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall('dataset-classifier-cat-dog-snake')


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

transform = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

data_dir = '/content/efficientnet-pytorch/dataset'

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory '{data_dir}' not found.")


dataset = datasets.ImageFolder(root=data_dir, transform=transform['train'])


dataset_size = len(dataset)
val_size = int(0.2 * dataset_size)
train_size = dataset_size - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

model = models.resnet18(pretrained=True)

num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'resnet18_model.pth')


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

Epoch 1/10, Loss: 0.2162


Validation Accuracy: 95.50%


Epoch 2/10: 100%|███████████████████████████████████████████████████| 75/75 [00:09<00:00,  7.57it/s]

Epoch 2/10, Loss: 0.1302


...

Epoch 9/10, Loss: 0.0757


Validation Accuracy: 95.33%


Epoch 10/10: 100%|██████████████████████████████████████████████████| 75/75 [00:10<00:00,  7.47it/s]

Epoch 10/10, Loss: 0.0882


Validation Accuracy: 95.50%
